From episcanpy use epi.tl.find_genes to find the peaks

# File opening(skip)

In [3]:
#edit based on your file path

file_path1 = "/Users/gajaj/Downloads/H_Kidney_Cancer_Chromium_Nuc_Iso_vs_SaltyEZ_vs_ComplexTissueDP_raw_feature_bc_matrix.tar.gz"
output_dir1 = "/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome"

In [4]:
with tarfile.open(file_path1, "r") as tar:
    tar.extractall(path=output_dir1)
    print("Files extracted successfully")

Files extracted successfully


In [5]:
#edit your file paths
input_file_barcodes1 = "Multiome/raw_feature_bc_matrix/barcodes.tsv.gz"
input_file_features1 = "Multiome/raw_feature_bc_matrix/features.tsv.gz"
input_file_matrix1 = "Multiome/raw_feature_bc_matrix/matrix.mtx.gz"

In [6]:
#edit your file paths
output_file_barcodes1 = "Multiome/raw_feature_bc_matrix/barcodes.tsv"
output_file_features1 = "Multiome/raw_feature_bc_matrix/features.tsv"
output_file_matrix1 = "Multiome/raw_feature_bc_matrix/matrix.mtx"


In [7]:
import gzip

with gzip.open(input_file_barcodes1, "rt") as gz_file:  
    with open(output_file_barcodes1, "w") as out_file:
        out_file.write(gz_file.read())

In [8]:
with gzip.open(input_file_features1, "rt") as gz_file:  
    with open(output_file_features1, "w") as out_file:
        out_file.write(gz_file.read())

In [9]:
with gzip.open(input_file_matrix1, "rt") as gz_file:  
    with open(output_file_matrix1, "w") as out_file:
        out_file.write(gz_file.read())

# Multiome

In [2]:
!pip install pyranges

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 13.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/772.9 kB ? eta -:--:--
   --------------------------------------- 772.9/772.9 kB 16.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 10.2 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import pandas as pd
import tarfile
import os
import pyranges as pr

In [2]:
import muon as mu

Read the multiome data

In [3]:
data_dir ='Multiome'
mdata = mu.read_10x_h5(os.path.join(data_dir, 'H_Kidney_Cancer_Chromium_Nuc_Iso_vs_SaltyEZ_vs_ComplexTissueDP_filtered_feature_bc_matrix.h5'))

c:\Users\gajaj\anaconda3\envs\preprocessing\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Added `interval` annotation for features from Multiome\H_Kidney_Cancer_Chromium_Nuc_Iso_vs_SaltyEZ_vs_ComplexTissueDP_filtered_feature_bc_matrix.h5


c:\Users\gajaj\anaconda3\envs\preprocessing\lib\site-packages\mudata\_core\mudata.py:477: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


In [5]:
mdata.var_names_make_unique()
mdata

MuData object with n_obs × n_vars = 22772 × 103751
  var:	'gene_ids', 'feature_types', 'genome', 'interval'
  2 modalities
    rna:	22772 x 36601
      var:	'gene_ids', 'feature_types', 'genome', 'interval'
    atac:	22772 x 67150
      var:	'gene_ids', 'feature_types', 'genome', 'interval'

In [6]:
atac = mdata.mod['atac']

Load the preprocessed data

In [7]:
csv_gene_ids = pd.read_csv('/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome/Barcodes_Gene_ids.tsv')  # Replace with the actual file path
preprocessed_data_list = csv_gene_ids['gene_ids'].tolist() 
len(preprocessed_data_list)

69144

In [8]:
# Separate into two lists
prepr_genes = [item for item in preprocessed_data_list if item.startswith('ENSG')]
prepr_peaks = [item for item in preprocessed_data_list if not item.startswith('ENSG')]

In [9]:
len(prepr_genes) + len(prepr_peaks)

69144

In [10]:
print(len(prepr_genes))
print(len(prepr_peaks))

2000
67144


Filter the atac data based on the preprocessed peak data

In [11]:
atac.shape


(22772, 67150)

In [12]:
filtered_atac = atac[:, atac.var['gene_ids'].isin(prepr_peaks)]
filtered_atac.shape

(22772, 67144)

Load the dataset of gene positions created in `get_gene_position.r`

In [13]:
# Load gene regions dataset
genes_df = pd.read_csv("Multiome/gene_positions.csv")

In [14]:
genes_df.columns = ["seqnames", "start", "end", "gene_id"]
genes_df

,seqnames,start,end,gene_id
0,1,3069168,3438621,ENSG00000142611
1,1,5301928,5307394,ENSG00000284616
2,1,2403964,2413797,ENSG00000157911
3,1,5492978,5494674,ENSG00000260972
4,1,10054445,10054781,ENSG00000224340
...,...,...,...,...
78927,KI270717.1,15119,17798,ENSG00000307722
78928,KI270717.1,24947,37269,ENSG00000310401
78929,KI270720.1,3633,31375,ENSG00000302039
78930,KI270718.1,6913,19848,ENSG00000309831


Filter the genes to the preprocessed ones

In [15]:
filtered_genes = genes_df[genes_df['gene_id'].isin(prepr_genes)]
filtered_genes

,seqnames,start,end,gene_id
10,1,9292894,9369532,ENSG00000171621
35,1,9997206,10016021,ENSG00000162444
63,1,12166991,12209228,ENSG00000028137
139,1,56494761,56645301,ENSG00000162407
188,1,84925583,84997113,ENSG00000153898
...,...,...,...,...
78713,MT,10470,10766,ENSG00000212907
78714,MT,10760,12137,ENSG00000198886
78718,MT,12337,14148,ENSG00000198786
78721,MT,14747,15887,ENSG00000198727


In [16]:
# Get the list of gene_ids from the dataset
dataset_gene_ids = genes_df['gene_id'].tolist()

# Find elements in prepr_genes that are not in dataset_gene_ids
missing_genes = [gene for gene in prepr_genes if gene not in dataset_gene_ids]

# Print the result
print(missing_genes)
print(len(missing_genes))


['ENSG00000286863', 'ENSG00000235736', 'ENSG00000228255', 'ENSG00000238042', 'ENSG00000233005', 'ENSG00000271955', 'ENSG00000235491', 'ENSG00000236501', 'ENSG00000236886', 'ENSG00000237838', 'ENSG00000269028', 'ENSG00000243944', 'ENSG00000249988', 'ENSG00000286494', 'ENSG00000253766', 'ENSG00000254138', 'ENSG00000248533', 'ENSG00000287263', 'ENSG00000249203', 'ENSG00000272370', 'ENSG00000249335', 'ENSG00000253985', 'ENSG00000249984', 'ENSG00000251314', 'ENSG00000250284', 'ENSG00000227220', 'ENSG00000285875', 'ENSG00000228334', 'ENSG00000287523', 'ENSG00000253983', 'ENSG00000253988', 'ENSG00000280560', 'ENSG00000254420', 'ENSG00000255028', 'ENSG00000256287', 'ENSG00000258216', 'ENSG00000257322', 'ENSG00000237356', 'ENSG00000258473', 'ENSG00000259255', 'ENSG00000261190', 'ENSG00000259995', 'ENSG00000272736', 'ENSG00000261222', 'ENSG00000267702', 'ENSG00000276251', 'ENSG00000235111', 'ENSG00000277666']
48


Get chromosomes and regions from the atac

In [17]:
atac.var[['chrom', 'start', 'end']] = atac.var['interval'].str.split(':|-', expand=True)
peaks = atac.var[['chrom', 'start', 'end']].copy()
peaks

,chrom,start,end
chr1:9855-10676,chr1,9855,10676
chr1:181027-181582,chr1,181027,181582
chr1:629494-630387,chr1,629494,630387
chr1:633523-634601,chr1,633523,634601
chr1:778319-779215,chr1,778319,779215
...,...,...,...
KI270713.1:21530-22452,KI270713.1,21530,22452
KI270713.1:26160-26918,KI270713.1,26160,26918
KI270713.1:28410-29017,KI270713.1,28410,29017
KI270713.1:32755-33461,KI270713.1,32755,33461


Remove 'chr' prefix in the chromosome names

In [18]:
# Remove 'chr' prefix if it exists
peaks["chrom"] = peaks["chrom"].str.replace("^chr", "", regex=True)

# Display the modified DataFrame
peaks


,chrom,start,end
chr1:9855-10676,1,9855,10676
chr1:181027-181582,1,181027,181582
chr1:629494-630387,1,629494,630387
chr1:633523-634601,1,633523,634601
chr1:778319-779215,1,778319,779215
...,...,...,...
KI270713.1:21530-22452,KI270713.1,21530,22452
KI270713.1:26160-26918,KI270713.1,26160,26918
KI270713.1:28410-29017,KI270713.1,28410,29017
KI270713.1:32755-33461,KI270713.1,32755,33461


Rename columns of gene dataset

In [19]:
genes_df = genes_df.rename(columns={
    "seqnames": "Chromosome",  # Rename seqnames to Chromosome
    "start": "Start",          # Rename start to Start
    "end": "End"               # Rename end to End
})
genes_df

,Chromosome,Start,End,gene_id
0,1,3069168,3438621,ENSG00000142611
1,1,5301928,5307394,ENSG00000284616
2,1,2403964,2413797,ENSG00000157911
3,1,5492978,5494674,ENSG00000260972
4,1,10054445,10054781,ENSG00000224340
...,...,...,...,...
78927,KI270717.1,15119,17798,ENSG00000307722
78928,KI270717.1,24947,37269,ENSG00000310401
78929,KI270720.1,3633,31375,ENSG00000302039
78930,KI270718.1,6913,19848,ENSG00000309831


Create a buffer on gene regions

In [20]:
genes_df['Start'] = genes_df['Start'] - 2000
genes_df['End'] = genes_df['End'] + 2000
genes_df

,Chromosome,Start,End,gene_id
0,1,3067168,3440621,ENSG00000142611
1,1,5299928,5309394,ENSG00000284616
2,1,2401964,2415797,ENSG00000157911
3,1,5490978,5496674,ENSG00000260972
4,1,10052445,10056781,ENSG00000224340
...,...,...,...,...
78927,KI270717.1,13119,19798,ENSG00000307722
78928,KI270717.1,22947,39269,ENSG00000310401
78929,KI270720.1,1633,33375,ENSG00000302039
78930,KI270718.1,4913,21848,ENSG00000309831


Create a PyRange object from gene dataset

In [21]:
genes_pr = pr.PyRanges(genes_df)
genes_pr

,Chromosome,Start,End,gene_id
0,1,3067168,3440621,ENSG00000142611
1,1,5299928,5309394,ENSG00000284616
2,1,2401964,2415797,ENSG00000157911
3,1,5490978,5496674,ENSG00000260972
4,1,10052445,10056781,ENSG00000224340
...,...,...,...,...
78927,Y,9896786,9910137,ENSG00000309435
78928,Y,6264593,6271275,ENSG00000238235
78929,Y,6264602,6277213,ENSG00000296934
78930,Y,6326052,6345323,ENSG00000223744


Rename columns of peaks dataset

In [22]:
peaks = peaks.rename(columns={"chrom": "Chromosome", "start": "Start", "end": "End"})
peaks

,Chromosome,Start,End
chr1:9855-10676,1,9855,10676
chr1:181027-181582,1,181027,181582
chr1:629494-630387,1,629494,630387
chr1:633523-634601,1,633523,634601
chr1:778319-779215,1,778319,779215
...,...,...,...
KI270713.1:21530-22452,KI270713.1,21530,22452
KI270713.1:26160-26918,KI270713.1,26160,26918
KI270713.1:28410-29017,KI270713.1,28410,29017
KI270713.1:32755-33461,KI270713.1,32755,33461


Create a PyRange object for peaks

In [23]:
peaks_pr = pr.PyRanges(peaks)
peaks_pr

,Chromosome,Start,End
0,1,9855,10676
1,1,181027,181582
2,1,629494,630387
3,1,633523,634601
4,1,778319,779215
...,...,...,...
67145,Y,56867424,56868313
67146,Y,56869540,56870326
67147,Y,56870767,56871793
67148,Y,56873588,56874343


Find overlaps between regions (start_b and end_b are from genes)

In [24]:
# Find overlaps
overlaps = peaks_pr.join(genes_pr)

# Convert overlaps to DataFrame for easier manipulation
overlap_df = overlaps.df
overlap_df

,Chromosome,Start,End,Start_b,End_b,gene_id
0,1,9855,10676,9121,26894,ENSG00000290825
1,1,9855,10676,10010,15670,ENSG00000223972
2,1,181027,181582,87295,183719,ENSG00000241860
3,1,181027,181582,144386,201861,ENSG00000310528
4,1,181027,181582,178931,186937,ENSG00000308415
...,...,...,...,...,...,...
92630,Y,11332807,11333567,11330329,11335595,ENSG00000258991
92631,Y,11333695,11334440,11330329,11335595,ENSG00000258991
92632,Y,11333695,11334440,11333627,11339693,ENSG00000270455
92633,Y,56855391,56856232,56853244,56857488,ENSG00000235857


In [27]:
overlap_df = overlap_df.rename(columns={"Start_b": "Start_gene", "Start": "Start_peak","End": "End_peak", "End_b": "End_gene"})
overlap_df

,Chromosome,Start_peak,End_peak,Start_gene,End_gene,gene_id
0,1,9855,10676,9121,26894,ENSG00000290825
1,1,9855,10676,10010,15670,ENSG00000223972
2,1,181027,181582,87295,183719,ENSG00000241860
3,1,181027,181582,144386,201861,ENSG00000310528
4,1,181027,181582,178931,186937,ENSG00000308415
...,...,...,...,...,...,...
92630,Y,11332807,11333567,11330329,11335595,ENSG00000258991
92631,Y,11333695,11334440,11330329,11335595,ENSG00000258991
92632,Y,11333695,11334440,11333627,11339693,ENSG00000270455
92633,Y,56855391,56856232,56853244,56857488,ENSG00000235857


Gene regions that are fully contained in the peak regions

In [25]:
# Find genes fully contained within peaks
contained_overlaps = peaks_pr.join(genes_pr, how="containment")

# Convert the results to a DataFrame for easier manipulation
contained_df = contained_overlaps.df

# Display the resulting DataFrame
contained_df


,Chromosome,Start,End,Start_b,End_b,gene_id
0,1,9855,10676,9121,26894,ENSG00000290825
1,1,181027,181582,87295,183719,ENSG00000241860
2,1,181027,181582,144386,201861,ENSG00000310528
3,1,181027,181582,178931,186937,ENSG00000308415
4,1,181027,181582,180146,203244,ENSG00000310527
...,...,...,...,...,...,...
87375,Y,11323126,11323733,11319557,11324823,ENSG00000259029
87376,Y,11332807,11333567,11330329,11335595,ENSG00000258991
87377,Y,11333695,11334440,11330329,11335595,ENSG00000258991
87378,Y,11333695,11334440,11333627,11339693,ENSG00000270455


### Create a tsv file

In [28]:
# Save as a TSV file
overlap_df.to_csv("overlap_genes_peaks.tsv", sep="\t", index=False)

# Confirm file creation
print("TSV file created successfully!")

TSV file created successfully!
